A ideia é desenvolver um algoritmo que seja capaz de prever informações a respeito da cotação do ouro em função do dia e das informações da cotação do dia anterior

Link do dataset: https://www.kaggle.com/datasets/guillemservera/precious-metals-data

In [6]:
import pandas as pd

df = pd.read_csv('gold.csv')

In [ ]:
!pip install -U kaleido

In [ ]:
# Gráfico de candlesticks

import plotly.graph_objects as go
import kaleido

grafico = go.Figure(go.Candlestick(x=df.date, open=df.open, high=df.high, low=df.low, close=df.close))
grafico.update_layout(xaxis_rangeslider_visible=False)  # remove range slider
grafico.show()

# grafico.write_image('candlestick.jpeg')

In [ ]:
import numpy as np

print(df.isnull().sum())  # verificando dados faltantes

# verificando dados negativos ou infinitos
for c in df.columns[1:]:
  for d in range(len(df[c])):
    if df[c][d] < 0 or df[c][d] == np.inf:
      print(f'Valor negativo encontrado na linha {d} coluna {c}')

In [11]:
# Normalizando dados

from sklearn.preprocessing import MaxAbsScaler, OrdinalEncoder

array = np.array(df.date).reshape(-1, 1)
df.date = OrdinalEncoder().fit_transform(array)

escalonador = MaxAbsScaler()
escalonador.fit(df)
dados = escalonador.transform(df)

In [ ]:
# Definição de entradas e saídas

# removi a primeira linha da saída e a coluna de data
# removi shiftei a coluna de data da saída e removi a última linha

entradas = dados.copy()

for c in range(len(entradas)):
  try:
    entradas[c][0] = entradas[c+1][0]
  except:
    print('Shifted Finalizado')

entradas = entradas[0:len(entradas) - 1]
saidas = dados.copy()[1:, 1:]

In [13]:
# Separação dos dados

from sklearn.model_selection import train_test_split as tts

intreino, inteste, outtreino, outteste = tts(entradas, saidas, shuffle=False, train_size=0.90)

In [ ]:
# Treinando rede neural

from sklearn.neural_network import MLPRegressor

nn = MLPRegressor(hidden_layer_sizes = [6, 5], activation='identity', max_iter = 10000)
nn.fit(intreino, outtreino)

outpredito = nn.predict(inteste)

# Avaliando resultados

from sklearn.metrics import max_error, mean_squared_error, mean_absolute_error, r2_score

print('Erro máximo:', max_error(outteste[0:, 3], outpredito[0:, 3]))
print('Erro médio quadrático:', mean_squared_error(outteste[0:, 3], outpredito[0:, 3]))
print('Erro médio absoluto:', mean_absolute_error(outteste[0:, 3], outpredito[0:, 3]))

In [ ]:
# Comparando preço de fechamento real com o predito

import matplotlib.pyplot as plt

plt.plot(inteste[0:, 0], outteste[0:, 3], 'r')
plt.plot(inteste[0:, 0], outpredito[0:, 3], 'g')

plt.title('Preço de Fechamento')
plt.xlabel('Tempo (normalizado)')
plt.ylabel('Preço (normalizado)')

plt.savefig('cotacao_fechamento.jpg', bbox_inches='tight')
plt.show()